In [ ]:
import numba
import numpy as np

def gather_from_detector():
    return np.random.random((1000, 1000))

@numba.jit
def smooth(x):
    out = np.empty_like(x)
    for i in range(1, x.shape[0] - 1):
        for j in range(1, x.shape[1] - 1):
            out[i, j] = (x[i + -1, j + -1] + x[i + -1, j + 0] + x[i + -1, j + 1] +
                         x[i +  0, j + -1] + x[i +  0, j + 0] + x[i +  0, j + 1] +
                         x[i +  1, j + -1] + x[i +  1, j + 0] + x[i +  1, j + 1]) // 9

    return out

def save(x, filename):
    pass

In [ ]:
from dask.distributed import LocalCluster, Client, fire_and_forget

cluster = LocalCluster()
client = Client(cluster)
client

In [ ]:
for i in range(100):
    img = client.submit(gather_from_detector, pure=False)
    img = client.submit(smooth, img)
    img = client.submit(np.fft.fft2, img)
    future = client.submit(save, img, "file-" + str(i) + "-.dat")
    fire_and_forget(future)

In [ ]:
import asyncio

async def gather():
    while True:
        img = client.submit(gather_from_detector, pure=False)
        img = client.submit(smooth, img)
        img = client.submit(np.fft.fft2, img)
        future = client.submit(save, img, "file-" + str(i) + "-.dat")
        fire_and_forget(future)
        await asyncio.sleep(0.01)
        
asyncio.ensure_future(gather())

In [ ]:
cluster.scale(6)